1. joinujemy informacje o ofercie z DF offer do informacji o kliencie z DF df2
2. robimy model gdzie labelem jest cliKed (kurwa)
3. żeby zrobić predykcje na nowym kliecie crossjoinujemy go z ofertami i robimy predykcje
4. oferta wybrana dla klienta to ta dla której wyjdzie największe prawdopodobieństwo kliknięcia

#### ew. jakieś grupowanie klientów

do zrobienia:
- join ofert do df
- ogarnięcie stringów
- OHE danych o ofercie
- jakaś normalizacja
- budujemy XGboosta, jakąś sieć, KNN
- porównanie modeli po accuracy, precision, recall
- XAI na jednej zmiennej albo dwóch
- przygotowanie wersji do pseudo-produkcji
- prezka

In [2]:
import pandas as pd
import numpy as np

In [61]:
df = pd.read_json('/Users/mkunstler/Downloads/lot_ub_uw_personalization_task/context_history.json')

In [43]:
df.head()

,context,cliked,request_dttm,offer_id
0,"{'booking': {'param1': 3, 'param2': 0, 'param3...",0,2023-02-20 00:54:42,OFFER_C_1_2_3_4_5
1,"{'booking': {'param1': 1, 'param2': 0, 'param3...",1,2023-02-20 00:54:42,OFFER_C_1_2_3_4_5
2,"{'booking': {'param1': 2, 'param2': 1, 'param3...",0,2023-02-20 00:54:42,OFFER_A_5_2_4_1_3
3,"{'booking': {'param1': 3, 'param2': 0, 'param3...",0,2023-02-20 00:54:42,OFFER_A_1_2_3_4_5
4,"{'booking': {'param1': 2, 'param2': 2, 'param3...",0,2023-02-20 00:54:42,OFFER_A_5_2_4_1_3


In [29]:
offer = pd.read_json('/Users/mkunstler/Downloads/lot_ub_uw_personalization_task/offer_features.json')

offer = pd.concat([offer.drop(['features'], axis=1), pd.DataFrame(offer['features'].to_list(), columns = ['Fet']).Fet.apply(pd.Series)], axis=1)

offer.head()

,id,schema,top_1_product,top_2_product,top_3_product,top_4_product,top_5_product,is_banner,products_offered,cta_number
0,OFFER_A_1_2_3_4_5,A,1,2,3,4,5,True,5,2
1,OFFER_B_1_2_3_4_5,B,1,2,3,4,5,False,5,2
2,OFFER_C_1_2_3_4_5,C,1,2,3,4,5,False,5,5
3,OFFER_D_1_2_3_4_5,D,1,2,3,4,5,True,3,2
4,OFFER_E_1_2_3_4_5,E,1,2,3,4,5,False,5,1


In [116]:
offer

,id,schema,top_1_product,top_2_product,top_3_product,top_4_product,top_5_product,is_banner,products_offered,cta_number
0,OFFER_A_1_2_3_4_5,A,1,2,3,4,5,True,5,2
1,OFFER_B_1_2_3_4_5,B,1,2,3,4,5,False,5,2
2,OFFER_C_1_2_3_4_5,C,1,2,3,4,5,False,5,5
3,OFFER_D_1_2_3_4_5,D,1,2,3,4,5,True,3,2
4,OFFER_E_1_2_3_4_5,E,1,2,3,4,5,False,5,1
5,OFFER_A_2_1_3_4_5,A,2,1,3,4,5,True,5,2
6,OFFER_B_2_1_3_4_5,B,2,1,3,4,5,False,5,2
7,OFFER_C_2_1_3_4_5,C,2,1,3,4,5,False,5,5
8,OFFER_D_2_1_3_4_5,D,2,1,3,4,5,True,3,2
9,OFFER_E_2_1_3_4_5,E,2,1,3,4,5,False,5,1


In [62]:
context_table = pd.DataFrame(df['context'].to_list(), columns = ['booking', 'leg', "param1", "param2", "param3"])

booking = pd.DataFrame.from_records(context_table['booking'])
booking.columns = [x + "_booking" for x in booking.columns]

leg = pd.DataFrame.from_records(context_table['leg'])
leg.columns = [x + "_leg" for x in leg.columns]

context_table.drop(['booking', 'leg'], axis=1)
context_table.columns = [x + "_general" for x in context_table.columns]

In [73]:
df.drop(['context'], axis=1)
df2 = pd.concat([context_table, booking, leg, df], axis=1)

df2 = df2.drop(['booking_general', 'leg_general', 'context'], axis=1)

In [74]:
df2.head()

,param1_general,param2_general,param3_general,param1_booking,param2_booking,param3_booking,param4_booking,param5_booking,param6_booking,param7_booking,...,param8_leg,param9_leg,param10_leg,param12_leg,param13_leg,param14_leg,param15_leg,cliked,request_dttm,offer_id
0,1,0,d7202e9912cb41ddd9df71b6fe671bd0,3,0,0,2,3,22,4,...,1,0,2,91befdaafbf21c75a63fb0fb6d40d2d5,a85ee8a49e710f691d9688c3b68dbbc8034d0e2f6595b6...,c6b57d1bbfccb4c262bce0e014247ec5,9c0c9f48e51d24032bacb9a2daac3697,0,2023-02-20 00:54:42,OFFER_C_1_2_3_4_5
1,1,0,f07bc56e16b90ae5bf58b0f7c63aeaed,1,0,0,2,1,0,2,...,0,204,1,91befdaafbf21c75a63fb0fb6d40d2d5,a85ee8a49e710f691d9688c3b68dbbc8034d0e2f6595b6...,a49c05cb72f7f0abb3ed34a15fbe877c,9c0c9f48e51d24032bacb9a2daac3697,1,2023-02-20 00:54:42,OFFER_C_1_2_3_4_5
2,1,0,d7202e9912cb41ddd9df71b6fe671bd0,2,1,0,2,3,40,2,...,1,0,2,91befdaafbf21c75a63fb0fb6d40d2d5,a85ee8a49e710f691d9688c3b68dbbc8034d0e2f6595b6...,a49c05cb72f7f0abb3ed34a15fbe877c,9c0c9f48e51d24032bacb9a2daac3697,0,2023-02-20 00:54:42,OFFER_A_5_2_4_1_3
3,1,0,d7202e9912cb41ddd9df71b6fe671bd0,3,0,0,2,3,49,2,...,1,0,2,91befdaafbf21c75a63fb0fb6d40d2d5,a85ee8a49e710f691d9688c3b68dbbc8034d0e2f6595b6...,a49c05cb72f7f0abb3ed34a15fbe877c,9c0c9f48e51d24032bacb9a2daac3697,0,2023-02-20 00:54:42,OFFER_A_1_2_3_4_5
4,1,0,d7202e9912cb41ddd9df71b6fe671bd0,2,2,0,2,4,17,2,...,0,228,2,91befdaafbf21c75a63fb0fb6d40d2d5,a85ee8a49e710f691d9688c3b68dbbc8034d0e2f6595b6...,a49c05cb72f7f0abb3ed34a15fbe877c,9c0c9f48e51d24032bacb9a2daac3697,0,2023-02-20 00:54:42,OFFER_A_5_2_4_1_3


In [104]:
# Count the number of unique values in each column
unique_counts = df2.nunique()
unique_counts

param1_general         2
param2_general         1
param3_general        12
param1_booking        68
param2_booking         8
param3_booking         4
param4_booking         6
param5_booking        66
param6_booking        98
param7_booking         8
param8_booking        85
param9_booking        12
param10_booking       79
param11_booking       84
param12_booking        5
param13_booking        3
param14_booking       82
param15_booking        5
param16_booking       47
param1_leg            24
param2_leg            24
param3_leg            22
param4_leg             8
param5_leg            53
param6_leg             2
param7_leg            22
param8_leg             2
param9_leg          4789
param10_leg            6
param12_leg           47
param13_leg           47
param14_leg           78
param15_leg           78
cliked                 2
request_dttm       13735
offer_id              29
schema                 5
dtype: int64

In [103]:
df2['cliked'].value_counts()

0    496512
1    337555
Name: cliked, dtype: int64

In [78]:
df2['schema'] = df2['offer_id'].apply(lambda x: x.split("_")[1])

In [97]:
df3 = df2.loc[df2['cliked'] == 1]

In [105]:
features = ['param1_general', 'param2_general', 'param1_booking',
       'param2_booking', 'param3_booking', 'param4_booking', 'param5_booking',
       'param6_booking', 'param7_booking']
target = ['offer_id']

X_df = df3[features]
y_df = df3[target]

In [106]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size = 0.3, random_state = 30)

# define parameters' range
tree_param = [{'criterion': ['entropy'], 'max_depth': [15], 
              'min_samples_leaf': [5]}]

# perform fitting with cross-validation 
for metric in ['accuracy']:
    for n_folds in [3]:
        clf_gini = DecisionTreeClassifier()
        gini_grs = GridSearchCV(clf_gini, tree_param, scoring=metric, cv=n_folds, verbose=1)
        gini_grs.fit(X_train, y_train)
        print('Number of folds =' , n_folds,',', metric, gini_grs.bestscore, gini_grs.bestparams)


Fitting 3 folds for each of 1 candidates, totalling 3 fits


AttributeError: 'GridSearchCV' object has no attribute 'bestscore'

In [107]:
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


dtree_model = RandomForestClassifier().fit(X_train, y_train)
dtree_predictions = dtree_model.predict(X_test)

/var/folders/dc/k7crvlf16t786rmzsdp6xqs00000gn/T/ipykernel_79469/3231286087.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  dtree_model = RandomForestClassifier().fit(X_train, y_train)


In [108]:
confusion_matrix(y_test, dtree_predictions)

array([[  43,    0,   14,   17,   14,   16,    5,    8,  157,   14,   40,
          27,   31,    3,   28,    3,  149,  137,   24,    0,   32,   35,
        2188,   41,    3,   61,   16,   28,   21],
       [   4,    6,    5,    3,    4,    9,    1,    7,   25,    4,   15,
           7,    1,    0,    8,    0,   39,   17,    5,    2,   23,   10,
         555,    7,    4,    6,    4,   11,    8],
       [   8,    4,   58,   19,   19,   23,   13,    2,  140,   14,   53,
          45,   30,    1,   45,   10,  144,  172,   25,    3,   43,   35,
        2156,   41,    4,   72,   12,   28,   29],
       [   8,    3,   10,   67,   10,   14,   15,    6,  155,   21,   46,
          26,   39,    1,   49,   11,  157,  196,   32,   10,   47,   24,
        2504,   46,    2,  100,   20,   28,   20],
       [   6,    1,   12,   19,   44,   20,   18,    3,  136,    9,   38,
          33,   27,    2,   45,    2,  162,  188,   22,    1,   28,   25,
        1948,   36,    5,   86,    6,   27,   17],
     

In [109]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, dtree_predictions)

0.08227754352355654

In [110]:
1/29

0.034482758620689655

In [115]:
df3.offer_id.value_counts()

OFFER_D_5_2_4_1_3    26148
OFFER_B_3_1_4_2_5    18584
OFFER_C_5_1_2_3_4    18471
OFFER_C_5_2_4_1_3    18121
OFFER_B_5_1_2_3_4    15176
OFFER_B_5_2_4_1_3    14908
OFFER_D_3_1_4_2_5    14429
OFFER_E_3_1_4_2_5    14307
OFFER_C_3_1_4_2_5    13712
OFFER_E_5_2_4_1_3    13391
OFFER_C_1_2_3_4_5    12804
OFFER_E_5_1_2_3_4    12771
OFFER_E_1_2_3_4_5    12411
OFFER_A_4_1_2_3_5    12368
OFFER_D_4_1_2_3_5    12099
OFFER_A_5_2_4_1_3    11900
OFFER_B_1_2_3_4_5    11194
OFFER_E_4_1_2_3_5    11041
OFFER_D_1_2_3_4_5    11009
OFFER_A_3_1_4_2_5    10709
OFFER_A_1_2_3_4_5    10692
OFFER_B_4_1_2_3_5    10316
OFFER_A_5_1_2_3_4     9932
OFFER_C_4_1_2_3_5     8842
OFFER_A_2_1_3_4_5     2694
OFFER_D_2_1_3_4_5     2654
OFFER_B_2_1_3_4_5     2433
OFFER_E_2_1_3_4_5     2349
OFFER_C_2_1_3_4_5     2090
Name: offer_id, dtype: int64